## Recherche par quadrillage de la profondeur et du taux d'apprentissage pour DIST_REL_C_01

Le modèle DIST_REL_C_01 offre des résultats prometteurs (cf. notebook 11.1), nous allons donc tenter de le perfectionner et d'approfondir l'apprentisssage (le modèle apprenait encore lorsqu'on a stoppé l'apprentissage).

Pour cela, nous allons dans un premier temps chercher par quadrillage le meilleur couple profondeur/taux d'apprentissage. Il s'agit en effet des paramètres influant le plus sur la rapidité de l'apprentissage. Une fois que nous aurons trouvé le couple qui satisfaira au mieux les critères de rapidité et d'efficacité, nous l'utiliserons pour optimiser le modèle, et comme base pour les modèles qui suivront.

Nous allons donc d'une part étudier les résultats de la recherche par quadrillage via Scikit-Learn, et d'autre part les courbes d'apprentissage via Tensorboard.

#### Chemin des fichiers

In [1]:
train_prepared_input_loc = "../data/DIST_REL_C_RNN/train_set_prepared_input.h5"
train_labels_loc = "../data/DIST_REL_C_RNN/train_set_labels.h5"

"""
train_prepared_input_loc =  "../data/DIST_REL_C_RNN/minimal_prepared_input.h5"
train_labels_loc = "../data/DIST_REL_C_RNN/minimal_labels.h5"
"""

models_loc = "../models/DIST_REL_C_01/11.2/"
logs_loc = "../models/DIST_REL_C_01/11.2/"

### Définition des fonctions de coût et de validation

#### RMSE (coût)

In [2]:
def rmse(pred, targets):
    with tf.name_scope("rmse_loss"):
        return tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, targets)), name="rmse")

#### Fonction d'évaluation des performances (opposé du RMSE)


In [3]:
def rmse_valid(pred, targets, inputs):
    with tf.name_scope("rmse_validation"):
        return -rmse(pred, targets)

## Création du RN

In [4]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import tflearn as tfl
import math


def creer_RN(epsilon=1e-4, learning_rate=0.001, dropout_val=0.98, stddev_init=0.001,
             hidden_act='elu', outlayer_act='linear', weight_decay=0.001, width=870, depth=3,
             validation_fun=rmse_valid, cost_fun=rmse):

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
        
    # On créé l'input du RN
    network = input_data(shape=[None, 870], name='input')
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit,
                                  weight_decay=weight_decay)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 1, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=cost_fun, metric=validation_fun, name='target')
            
    return network

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


## Entraînement des modèles

#### Classe permettant d'interfacer les modèles TFLearn avec Scikit-Learn

In [1]:
from sklearn.base import BaseEstimator, RegressorMixin
import tensorflow as tf
import tflearn as tfl

class RNRegressor(BaseEstimator, RegressorMixin):
    """ Wrapper autour de tflearn que l'on utilise pour utiliser les modèles depuis Scikit-Learn
    Inspiré de http://danielhnyk.cz/creating-your-own-estimator-scikit-learn/ """
    
    
    def __init__(self, log_dir="../models/DIST_REL_C_01/grid_logs11.2" ,learning_rate=0.001, epsilon=1e-4, dropout=0.98, 
                 stddev_init=0.001, hidden_act="elu", outlayer_act='linear', weight_decay=0.001, 
                 width=870, depth=3, batch_size=10000, epochs=300, loss_fun=rmse,
                 score_fun=rmse_valid):
        
        self.log_dir = log_dir
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.dropout = dropout
        self.stddev_init = stddev_init
        self.hidden_act = hidden_act
        self.outlayer_act = outlayer_act
        self.weight_decay = weight_decay
        self.width = width
        self.depth = depth
        self.batch_size = batch_size
        self.epochs = epochs
        self.loss_fun = loss_fun
        self.score_fun = score_fun
        
        
    def fit(self, X, y=None):
        
        tf.reset_default_graph()
        tfl.init_graph(num_cores=4,gpu_memory_fraction=0.25)


        # On créé le réseau 
        network = creer_RN(learning_rate=self.learning_rate, epsilon=self.epsilon, 
                                      dropout_val=self.dropout, stddev_init=self.stddev_init, 
                                      hidden_act=self.hidden_act, outlayer_act=self.outlayer_act,
                                      weight_decay=self.weight_decay, width=self.width, depth=self.depth,
                                      validation_fun=self.score_fun, cost_fun=self.loss_fun)

        # On créé le modèle
        self.model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_loc)

        model_name = ("lr"+str(self.learning_rate)+"eps"+str(self.epsilon)+"do"+str(self.dropout)+
                      "stddev_init"+str(self.stddev_init)+"hidact"+self.hidden_act+"out"+self.outlayer_act+
                      "wd"+str(self.weight_decay)+"w"+str(self.width)+"d"+str(self.depth)+"batchs"+
                      str(self.batch_size)+"epochs"+str(self.epochs))
        
        # Entraînement
        self.model.fit(X_inputs=X,Y_targets=y, batch_size=self.batch_size,
                  shuffle = False, snapshot_step=10000, validation_set=0.1,
                  show_metric=True, run_id=model_name, n_epoch=self.epochs)
        
        
        
    def predict(self, X, y=None):
        return self.model.predict(X, y)
    
    
    def score(self, X, y=None):
        return self.model.evaluate(X, y, batch_size=100)[0]

    
    

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


NameError: name 'rmse' is not defined

#### Définition d'une fonction faisant appel à GridSearchCV de scikit learn selon les paramètres du modèle passés en entrée. 


In [6]:
from sklearn.model_selection import GridSearchCV


def recherche_quadrillage(parametres, input_X, input_y):
    
    grid_search = GridSearchCV(estimator=RNRegressor(), param_grid=parametres, cv=2, n_jobs=3)
    
    grid_search.fit(input_X, input_y)
        
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    print(grid_search.best_estimator_.get_params())
    


#### Définition de la grille d'hyper-paramètres à tester et lancement de la recherche

In [7]:
import h5py
import numpy as np
import tensorflow as tf
from tflearn.data_utils import pad_sequences
import time

"""
parameters = {'learning_rate': [0.01, 0.02],
              'depth': [1, 2, 3]}

input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')

input_X = np.array(input_X_h5["inputs"])
input_X = pad_sequences(input_X, dtype="float32", maxlen=870)
input_X = input_X.reshape(-1, 870)

labels_y = np.array(labels_y_h5["targets"])
labels_y = labels_y.reshape(-1, 1)

total_start_time = time.time()
recherche_quadrillage(parameters, input_X, labels_y)
print("--- %s seconds ---" % (time.time() - total_start_time))"""


KeyboardInterrupt: 

##### Sortie

```
Best score: -27.560
Best parameters set:
{'learning_rate': 0.01, 'weight_decay': 0.001, 'epochs': 300, 'log_dir': '../models/DIST_REL_C_01/grid_logs11.2', 'loss_fun': <function rmse at 0x7f37b7ee2f28>, 'dropout': 0.98, 'depth': 3, 'outlayer_act': 'linear', 'batch_size': 10000, 'stddev_init': 0.001, 'epsilon': 0.0001, 'hidden_act': 'elu', 'score_fun': <function rmse_valid at 0x7f37b7ee9048>, 'width': 870}
--- 57304.7722120285 seconds ---
```

## Analyse des performances et nouvelle recherche

L'analyse des résultats de tensorboard comme de la recherche par quadrillage de Scikit-Learn montre que les meilleurs modèles sont ceux possédant trois couches et un taux d'apprentissage de 0.01. Pour optimiser encore l'apprentissage, nous allons maintenant effectuer une nouvelle recherche par quadrillage des paramètres de learning rate et de profondeur, avec des learning rates plus basses et des réseaux plus profonds.

#### Chemin des fichiers

In [2]:
models_loc = "../models/DIST_REL_C_01/11.2bis/"
logs_loc = "../models/DIST_REL_C_01/11.2bis/"

In [ ]:
parameters = [{'learning_rate': [0.01], 'depth': [4, 5]},
              {'learning_rate': [0.005, 0.001], 'depth': [3]},
              {'learning_rate': [0.005, 0.001, 0.008], 'depth': [4, 5, 6]}]


input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')

input_X = np.array(input_X_h5["inputs"])
input_X = pad_sequences(input_X, dtype="float32", maxlen=870)
input_X = input_X.reshape(-1, 870)

labels_y = np.array(labels_y_h5["targets"])
labels_y = labels_y.reshape(-1, 1)

total_start_time = time.time()
recherche_quadrillage(parameters, input_X, labels_y)
print("--- %s seconds ---" % (time.time() - total_start_time))